In [1]:
import numpy as np
import h5py

import re
import os

import copy

In [2]:
input_data_path = '../Step_3/'

In [3]:
number_of_electrodes  = np.array([0,1,1,1]) 
# [a,b,c,d] means that
# 4 electrodes need at least a spikes,
# 3 electrodes need at least b spikes,
# 2 electrodes need at least c spikes, and
# 1 electrode needs at least d spikes.
# Both stims of the standard experiment are considered.

In [4]:
DIV = 27 # This is the DIV that is being
run =  2 # This run is considered for the data analsysis

In [5]:
# Is the circuit independent or not. 
# 1: The circuit is independent. 
# 0: The circuit is connected to another circuit and must not be used
connected = { #  Circuit:  0  1  2  3  4  5  6  7  8  9  10  11  12  13  14
                      2: [ 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,  0,  1,  0,  0,  1],
                      3: [ 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,  1,  1,  1,  1,  1],
                      4: [ 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,  1,  1,  1,  1,  1],
                      6: [ 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,  1,  1,  1,  1,  0],
                      7: [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0,  0,  0,  0,  0],
                      8: [ 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,  1,  1,  1,  1,  1],
                      9: [ 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,  1,  1,  1,  1,  1],
                     10: [ 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,  1,  1,  1,  1,  1]
}

In [6]:
# Is a cell clump present in each of the nodes?
# 1: Each node has at least one clump 
# 0: At least one node has no cell clump. Therefore, the circuit must not be used
cells_present = { #  Circuit:  0  1  2  3  4  5  6  7  8  9  10  11  12  13  14
                          2: [ 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,  1,  0,  1,  1,  1],
                          3: [ 1, 1, 1, 1, 1, 0, 1, 1, 0, 1,  1,  0,  1,  1,  0],
                          4: [ 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,  1,  1,  1,  1,  1],
                          6: [ 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,  1,  1,  1,  1,  1],
                          7: [ 1, 1, 1, 0, 1, 1, 0, 1, 1, 0,  1,  1,  0,  1,  1],
                          8: [ 1, 1, 1, 1, 0, 1, 1, 0, 0, 0,  0,  1,  0,  1,  1],
                          9: [ 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,  1,  1,  1,  1,  1],
                         10: [ 0, 0, 0, 0, 0, 1, 0, 1, 1, 0,  0,  1,  0,  0,  0]
}

In [7]:
# # Is a cell clump present in each of the nodes?
# # 1: Each node has at least one clump 
# # 0: At least one node has no cell clump. Therefore, the circuit must not be used
# cells_present = { #  Circuit:  0  1  2  3  4  5  6  7  8  9  10  11  12  13  14
#                           2: [ 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,  1, 1,  1, 1,  1],
#                           3: [ 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,  1, 1,  1, 1,  1],
#                           4: [ 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,  1, 1,  1, 1,  1],
#                           6: [ 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,  1, 1,  1, 1,  1],
#                           7: [ 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,  1, 1,  1, 1,  1],
#                           8: [ 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,  1, 1,  1, 1,  1],
#                           9: [ 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,  1, 1,  1, 1,  1],
#                          10: [ 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,  1, 1,  1, 1,  1]
# }

In [8]:
filenames = []
for (_,_,files) in os.walk(input_data_path):
    for filename in files:
        f_no_numbers = re.sub(r'[0-9]+','',filename)
        if f_no_numbers == 'MEA__DIV_.h' and str(DIV) in filename:
            filenames.append(filename)

if len(filenames) != 8:
    print(filenames)
    assert False # You need to have a DIV that contains all 8 MEAs. At least one MEA does not exist for this DIV.

In [9]:
# Define, if activity properties are fullfilled
is_active = copy.deepcopy(cells_present)

for MEA in [2,3,4,6,7,8,9,10]:
    # Load the file
    f_in = h5py.File(input_data_path + 'MEA_' + str(MEA) + '_DIV_' + str(DIV) + '.h5','r')
    for circuit in range(15):

        # This counts, how many traces there are for each electrode
        count_per_elec = np.zeros(4)

        for pattern in [1,3]:
            try:
                group = f_in['Circuit_' + str(circuit) + '/Pattern_' + str(pattern) + '/Segment_' + str(run)]
                keys = np.array(list(group.keys()))
            except:
                print('MEA: ',MEA)
                print('Circuit: ',circuit)
                print('Pattern: ',pattern)
                assert False # There has been an error with this network (could not read the keys)
            # Remove any keys that are not ridge information
            keys = [key for key in keys if "Ridge_" in key]

            elecs = []
            for key in keys:
                elecs.append(group[key][0,2])

            count_per_elec += np.array([np.sum((np.array(elecs)==i)*1.) for i in range(4)])

        # If 1, than all for conditions for trace count have been fullfiled. Otherwise, it is 0.
        count_per_elec = np.sort(count_per_elec)
        circuit_is_good = (np.min((count_per_elec>=number_of_electrodes)*1))

        is_active[MEA][circuit] = circuit_is_good

    f_in.close()

In [10]:
# This saves the state for all circuits
circuit_mask = copy.deepcopy(cells_present)
for i in [2,3,4,6,7,8,9,10]:
    circuit_mask[i] = np.array(cells_present[i])*np.array(is_active[i])*np.array(connected[i])

In [11]:
circuit_mask

{2: array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 3: array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 4: array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]),
 6: array([1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0]),
 7: array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 8: array([1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1]),
 9: array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 10: array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])}

In [12]:
print("      Percentages      ")
print("-----------------------")
print("Independence:   ", np.round(np.sum([np.sum(connected[i]) for i in [2,3,4,6,7,8,9,10]])*100/(8*15),1), "%")
print("Enough neurons: ", np.round(np.sum([np.sum(cells_present[i]) for i in [2,3,4,6,7,8,9,10]])*100/(8*15),1), "%")
print("Useful activity:", np.round(np.sum([np.sum(is_active[i]) for i in [2,3,4,6,7,8,9,10]])*100/(8*15),1), "%")
print("Total:          ", np.round(np.sum([np.sum(circuit_mask[i]) for i in [2,3,4,6,7,8,9,10]])*100/(8*15),1), "%")

      Percentages      
-----------------------
Independence:    75.8 %
Enough neurons:  75.8 %
Useful activity: 20.0 %
Total:           14.2 %


In [13]:
f = h5py.File('../Step_4/step_4_percentages.h5','w')
for MEA in [2,3,4,6,7,8,9,10]:
    MEA_group = f.create_group('MEA_' + str(MEA))
    MEA_group.create_dataset(name='independence',  data=connected[MEA])
    MEA_group.create_dataset(name='enough_neurons',data=cells_present[MEA])
    MEA_group.create_dataset(name='is_active',data=is_active[MEA])
    MEA_group.create_dataset(name='total',data=circuit_mask[MEA])
f.close()